## Условия задачи:
<ul>
<li><b>Начальная популяция:</b> жадный выбор, начиная со случайного груза</li>
<li><b>Отбор особей для скрещивания:</b> выбор каждой особи пропорционально приспособленности (рулетка)</li>
<li><b>Скрещивание (кроссинговер) между выбранными особями. Каждая особь
скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка:</b> однородный (каждый бит от случайно выбранного родителя)</li>
<li><b>Мутация:</b> добавление 1 случайной вещи 10% особей</li>
<li><b>Формирование новой популяции (кол-во особей-константа):</b> замена своих родителей</li>
</ul>

In [265]:
import operator
import random
from IPython.display import clear_output

In [266]:
# путь к файлу с исходными данными
path = "src/18.txt"

data = []

# грузоподъемность рюкзака
maxW = 0
# ценность рюкзака
maxV = 0


fRow = True
with open(path, "r") as file:
    for i in file:
        # первая строка с параметрами рюкзака
        if (fRow):
            fRow = False
            maxW, maxV = i.split()
            # кастуем к integer
            maxW = int(maxW)
            maxV = int(maxV)
        # остальные строки с параметрами предметов
        else:
            data.append(i.split())

print("Грузоподъемность рюкзака: " + str(maxW))
print("Вместимость рюкзака: " + str(maxV))

Грузоподъемность рюкзака: 13000
Вместимость рюкзака: 12


In [267]:
# вес
w = []
# объем
v = []
# цэ - цЭнность :)
c = []

for i in range(len(data)):
    w.append(int(data[i][0]))
    v.append(float(data[i][1]))
    c.append(int(data[i][2]))
    
data = list(zip(w, v, c))

In [268]:
def paramsFromGenome(genome):
    w = 0
    v = 0
    c = 0
    for i in range(len(genome)):
        if(int(genome[i])):
            w += data[i][0]
            v += data[i][1]
            c += data[i][2]
    return {"genome": genome,
            "w": w,
            "v": v,
            "c": c}

In [269]:
def generateIndivid(item, itemArr):
    
    itemArr = sorted(itemArr, key = lambda x: (x[2]), reverse=True)
    individ = "0" * len(itemArr)
    w = 0
    v = 0
    c = 0
    
    while(item < len(itemArr) and w+itemArr[item][0] <= maxW and v+itemArr[item][1] <= maxV):
        w += itemArr[item][0]
        v += itemArr[item][1]
        c += itemArr[item][2]
        individ = individ[:(item)] + "1" + individ[(item + 1):]
        item += 1
    return {"genome": individ,
            "w": w,
            "v": v,
            "c": c}

In [270]:
def _validate_individ(individ):
    weight = individ['w']
    volume = individ['v']
    if (weight > maxW or volume > maxV):
        return False;
    return True

def selection(population, s=20):
    selectioner = int(len(population) * s / 100) 
    population = sorted(population, key = lambda i: i['c'],reverse=True)
    
    selected = []
    not_selected = []
    selected_counter = 0
    
    for individ in population:

        if (not _validate_individ(individ)):
            not_selected.append(individ)
        elif(selected_counter < selectioner):
            selected.append(individ)
            selected_counter += 1
        else:
            not_selected.append(individ)
            
    return selected, not_selected

In [271]:
def randomParents(start, stop, count):
    # Получаем неповторяющиеся случайные числа
    gamma = []
    for i in range(0, count):
        while True:
            item = random.randint(start, stop)
            if not gamma.count(item):
                gamma.append(item)
                yield item
                break

In [272]:
def sex(p1, p2):
    fChildGenome = ""
    sСhildGenome = ""
    for i in range(len(p1["genome"])):
        p = random.randint(0, 1)

        if(p):
            fChildGenome += p2["genome"][i]
            sСhildGenome += p1["genome"][i]
        else:
            fChildGenome += p1["genome"][i]
            sСhildGenome += p2["genome"][i]
    
    fChild = paramsFromGenome(fChildGenome)
    sChild = paramsFromGenome(sСhildGenome)
    
    return [fChild, sChild]

In [273]:
def reproduction(parentPopulation):
    randomQueue = list(randomParents(0, len(parentPopulation) - 1, len(parentPopulation)))

    newPopulation = []
    for i in range(0, len(randomQueue), 2):
        try:
            newPopulation = newPopulation + sex(parentPopulation[randomQueue[i]], parentPopulation[randomQueue[i + 1]])
        except:
            oldPopulation.append(parentPopulation[randomQueue[i]])
    return newPopulation

In [274]:
def mutation(population, pp=10):
    resPop = population
    individs = list(randomParents(0, len(population) - 1, int(len(population) / 10)))
    for i in individs:
        ones = []
        for ch in range(len(resPop[i]["genome"])):
            if (resPop[i]["genome"][ch] == "0"):
                ones.append(ch)
        mutated = ones[random.randint(0, len(ones) - 1)]
        resPop[i]["genome"] = resPop[i]["genome"][:(mutated)] + "1" + resPop[i]["genome"][(mutated + 1):]

    return resPop

## SCIENCE, BITCH

In [281]:
populationNumber = 200
population = []
for i in range(populationNumber):
    population.append(generateIndivid(random.randint(0, len(data)), data))

iterCount = 500

for i in range(iterCount):
    print("Итерация: " + str(i) + "/" + str(iterCount))
    
    parentPopulation, oldPopulation = selection(population)
    childPopulation = reproduction(parentPopulation)
    mutatedPopulation = mutation(childPopulation)
    #population = oldPopulation + mutatedPopulation
    population = mutatedPopulation + childPopulation + childPopulation + childPopulation + childPopulation
    
    clear_output()

population, oldPopulation = selection(population)
population = sorted(population, key = lambda i: i["c"],reverse=True)
print(population[0]["c"])
print(population[0]["v"])
print(population[0]["w"])

3961
11.7
12816


In [282]:
population = sorted(population, key = lambda i: i["c"],reverse=True)
population

[{'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111010100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000100',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000',
  'w': 12816,
  'v': 11.7,
  'c': 3961},
 {'genome': '001011111111111111000100000000